In [1]:
import fiftyone as fo
import fiftyone.zoo as foz

train_dataset = foz.load_zoo_dataset(
    "coco-2017", split="train", max_samples=5 )
val_dataset = foz.load_zoo_dataset(
    "coco-2017", split="validation", max_samples=5 )
test_dataset = foz.load_zoo_dataset(
    "coco-2017", split="test", max_samples=5 )


datasets = [train_dataset, test_dataset, val_dataset]

session = fo.launch_app(auto=False)

Found annotations at '/home/hubs-guide/fiftyone/coco-2017/raw/instances_train2017.json'
Sufficient images already downloaded
Existing download of split 'train' is sufficient
Loading existing dataset 'coco-2017-train-5'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Found annotations at '/home/hubs-guide/fiftyone/coco-2017/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading existing dataset 'coco-2017-validation-5'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Found test info at '/home/hubs-guide/fiftyone/coco-2017/raw/image_info_test2017.json'
Sufficient images already downloaded
Existing download of split 'test' is sufficient
Loading existing dataset 'coco-2017-test-5'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use
Session launched. Run `session.show()` to 

In [2]:
for dataset in datasets:
    dataset.export(
        export_dir="./dataset/" + dataset.name,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="ground_truth",  # The field containing annotations
        classes=dataset.default_classes,      # List of class names
    )

Directory './dataset/coco-2017-train-5' already exists; export will be merged with existing files
 100% |█████████████████████| 5/5 [15.0ms elapsed, 0s remaining, 332.9 samples/s] 
Directory './dataset/coco-2017-test-5' already exists; export will be merged with existing files
 100% |█████████████████████| 5/5 [8.2ms elapsed, 0s remaining, 610.1 samples/s] 
Directory './dataset/coco-2017-validation-5' already exists; export will be merged with existing files
 100% |█████████████████████| 5/5 [23.6ms elapsed, 0s remaining, 212.2 samples/s] 


In [3]:
from os import listdir
from ultralytics import YOLO
from ultralytics import settings

# Load a model
model = YOLO("./yolo_model/yolo11n.pt")

# Train the model
train_results = model.train(
    data="./dataset.yaml",  # path to dataset YAML
    epochs=1,  # number of training epochs
    device="CUDA",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

metrics = model.val()

filename = listdir("./dataset/" + test_dataset.name + "/images/val/")[0]
results = model("./dataset/" + test_dataset.name + "/images/val/" + filename)
results[0].save("./result.png")

New https://pypi.org/project/ultralytics/8.3.94 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.91 🚀 Python-3.10.8 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: task=detect, mode=train, model=./yolo_model/yolo11n.pt, data=./dataset.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=CUDA, workers=8, project=None, name=train23, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None

100%|██████████| 5.35M/5.35M [00:00<00:00, 13.8MB/s]


AMP: checks passed ✅


train: Scanning /home/hubs-guide/test_coco/dataset/coco-2017-train-5/labels/val.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]
val: Scanning /home/hubs-guide/test_coco/dataset/coco-2017-validation-5/labels/val.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]


Plotting labels to runs/detect/train23/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train23
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1     0.807G     0.8319      1.946      1.153         46        640: 100%|██████████| 1/1 [00:00<00:00,  1.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.32it/s]


                   all          5         47      0.613      0.604      0.692      0.467

1 epochs completed in 0.001 hours.
Optimizer stripped from runs/detect/train23/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train23/weights/best.pt, 5.5MB

Validating runs/detect/train23/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.10.8 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.12it/s]


                   all          5         47      0.613      0.604      0.692      0.466
                person          1          2      0.661          1      0.828      0.382
                   car          1          1          1          0          0          0
                 truck          1          1      0.254      0.508      0.497      0.249
             stop sign          1          2          1      0.947      0.995      0.829
                  bear          1          1      0.563          1      0.995      0.995
                 chair          2          6      0.744      0.488      0.577      0.409
          potted plant          2          3       0.42      0.667      0.683      0.488
                   bed          2          2      0.562        0.5      0.828      0.658
          dining table          1          1      0.332          1      0.995      0.249
                    tv          1          2      0.531          1      0.995      0.899
             microwav

KeyboardInterrupt: 

In [ ]:
print(len(results[0].boxes.cls))
print(len(results[0].boxes.conf))

4
4


In [ ]:
#дублирующие/структура